In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20180514_MO_Pax7_Laminin/wm/migration'

In [3]:
# Add experiment date here to apply to dataframe
expt_date = '20180514'
treatment = 'SMPD3MO'
result_type = 'Migration'

path = os.path.abspath('')+'/pax7migrationCSV/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    df['Treatment'], df['Dose'], df['Embryo'], df['Somites'], df['Mag'] = zip(*df['Image'].map(lambda x: x.split('_')))
    
    # Replace ROI names
    # Updated macro to avoid this step in future measurements
    df.replace(to_replace=['Cntl1', 'Cntl2', 'Cntl3', 'Cntl4', 'Expt1', 'Expt2', 'Expt3', 'Expt4'],
                            value=['Cntl', 'Cntl', 'Cntl', 'Cntl', 'Expt', 'Expt', 'Expt', 'Expt'], 
                            inplace=True)
    list_.append(df)

full_df = pd.concat(list_)

In [4]:
# Mean length values and group
area_meanLength = (full_df.groupby(['Embryo', 'ROI'])['Area', 'Length']).mean()

# Extract Areas, generate ratios
area_cntl = area_meanLength.xs('CntlArea', level='ROI')['Area'] 
area_expt = area_meanLength.xs('ExptArea', level='ROI')['Area'] 

# Generate ratios as Expt/Cntl
area_ratios = pd.DataFrame(area_expt / area_cntl)
area_ratios.columns = ['Expt/Cntl Area']

# Extract lengths, generate ratios
length_cntl = area_meanLength.xs('Cntl', level='ROI')['Length']
length_expt = area_meanLength.xs('Expt', level='ROI')['Length'] 

# Generate ratios as Expt/Cntl
length_ratios = pd.DataFrame(length_expt / length_cntl)
length_ratios.columns = ['Expt/Cntl Length']

In [5]:
# Combine processed values into single dataframe and output as csv file 'Results.csv'
area_cntl = pd.DataFrame(area_cntl)
area_cntl.columns = ['Cntl Area']
area_expt = pd.DataFrame(area_expt)
area_expt.columns = ['Expt Area']

length_cntl = pd.DataFrame(length_cntl)
length_cntl.columns = ['Cntl Length']
length_expt = pd.DataFrame(length_expt)
length_expt.columns = ['Expt Length']


results = pd.concat([area_cntl, area_expt, area_ratios, length_cntl, length_expt, length_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')

In [6]:
# Normalize all migration area values to mean of control group
norm_cntl = area_cntl/(area_cntl.mean())
norm_expt = area_expt/(area_expt.mean())
norm_areas = pd.concat([norm_cntl, norm_expt], axis=1, sort=False)
norm_areas['ID'] = expt_date + '_' + norm_areas.index
norm_areas.to_csv(expt_date + '_' + treatment + '_' + result_type + '_norm_areas.csv')
